In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Creating subagents

In [2]:
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """
    Calculate the square root of a number
    """

    return x ** 0.5


@tool
def square(x: float) -> float:
    """
    Calculate the square of number
    """

    return x ** 2

In [3]:
from langchain.agents import create_agent

# Create subagents

subagent_1 = create_agent(
    model="gpt-5-nano",
    tools=[square_root]
)

subagent_2 = create_agent(
    model="gpt-5-nano",
    tools=[square]
)

## Calling subagents

In [4]:
from langchain.messages import HumanMessage, SystemMessage

@tool
def call_subagent_1(x: float) -> float:
    """
    Call subagent 1 in order to calculate the square root of a number
    """
    
    user_query = HumanMessage(content=f"Calculate the square root of {x}")
    response = subagent_1.invoke(
        {"messages": [user_query]}
    )

    return response["messages"][-1].content


@tool
def call_subagent_2(x: float) -> float:
    """
    Call subagent 2 in order to calculate the square of a number
    """

    user_query = HumanMessage(content=f"Calculate the square of {x}")
    response = subagent_2.invoke(
        {"messages": [user_query]}
    )

    return response["messages"][-1].content

In [5]:
# Creating the main agent

system_prompt = SystemMessage(content="You are a helpful assistant who can call subagents to calculate the square root or a square of a number.")

main_agent = create_agent(
    model="gpt-5-nano",
    tools=[call_subagent_1, call_subagent_2],
    system_prompt=system_prompt
)

## Test

In [7]:
question = HumanMessage(content="What is the square root of 456?")

response = main_agent.invoke(
    {"messages": question}
)

In [8]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='2c179024-d8b8-4009-a87e-47645043f227'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 730, 'prompt_tokens': 203, 'total_tokens': 933, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 704, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CsvFDS0mS6NTJXRciUyV91s1x9aGf', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b75ac-71af-7663-8b22-45d36288fac4-0', tool_calls=[{'name': 'call_subagent_1', 'args': {'x': 456}, 'id': 'call_x8kThs1gsLfP1fKMzNEr72To', 'type': 'tool_call'}], usage_metadata={'input_tokens': 203, 'output_tokens': 730, 'to

https://smith.langchain.com/public/37b7e0de-e397-4577-9415-1c3dc394abf3/r

In [9]:
question = HumanMessage(content="What is the square of 10?")

response = main_agent.invoke(
    {"messages": [question]}
)

In [10]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is the square of 10?', additional_kwargs={}, response_metadata={}, id='00de9dd3-99f5-40b7-99cb-03ec1d8dfab0'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 283, 'prompt_tokens': 202, 'total_tokens': 485, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CsvJVCXCgBRCkTz8VoVdyvTMpyQQN', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b75b0-831b-7a43-9a50-72210a2e6c57-0', tool_calls=[{'name': 'call_subagent_2', 'args': {'x': 10}, 'id': 'call_3iZJFhbx6j0qUlgZOPQYV9Ab', 'type': 'tool_call'}], usage_metadata={'input_tokens': 202, 'output_tokens': 283, 'total_tok

https://smith.langchain.com/public/28701a82-95e3-4971-bb80-659d6c711602/r